In [ ]:
from keras import backend as K
K.clear_session()

from numba import cuda
cuda.select_device(0)
cuda.close()

In [10]:
from PIL import Image
import pandas as pd
import os
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers import Flatten

In [2]:
TRAIN_DIR = './imgs/train'

datagen = ImageDataGenerator(
        validation_split=0.3, rescale=1./255)


train_generator = datagen.flow_from_directory(
    TRAIN_DIR,  (224, 224), batch_size=20,
    subset='training'
    
)

val_generator = datagen.flow_from_directory(
    TRAIN_DIR, (224, 224),batch_size=20,
    subset='validation'
)

Found 15702 images belonging to 10 classes.
Found 6722 images belonging to 10 classes.


In [3]:
xceptionbase = Xception(weights='imagenet', include_top=False,
    input_shape=(224, 224, 3),
    pooling=None)
out1 = Flatten()(xceptionbase.output)
out1 = Dropout(0.5)(out1)
out1 = Dense(30, activation='relu')(out1)
out2 = Dense(10, activation="softmax")(out1)

xception1 = Model(inputs=xceptionbase.input, outputs=out2)
ad= SGD(lr=0.001,decay=1e-6, momentum=0.9)


xception1.compile(loss='categorical_crossentropy', optimizer=ad, metrics=['accuracy'])

print(xception1.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 111, 111, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 111, 111, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [4]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [5]:
hist = xception1.fit_generator(generator=train_generator, epochs=1, validation_data=val_generator, verbose=1)

Epoch 1/1
1571/1571 [==============================] - 320s 204ms/step - loss: 0.3550 - acc: 0.8808 - val_loss: 0.0488 - val_acc: 0.9874


In [9]:
from keras.preprocessing import image
from keras.applications.xception import preprocess_input
import numpy as np



img_path = './imgs/test/img_189.jpg'
img = image.load_img(img_path,target_size=(224, 224))
res = image.img_to_array(img)
res = np.expand_dims(res, axis=0)
res = preprocess_input(res)
preds = xception1.predict(res)
print(preds)

[[1.1092898e-09 3.6483227e-06 2.2150684e-04 6.4407533e-07 4.5940365e-10
  1.9889855e-08 2.4175051e-06 9.9592578e-01 3.8459317e-03 1.2621493e-07]]
